In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import calendar
import random
import seaborn as sns
import numpy as np
import itertools
import warnings
from itertools import groupby
warnings.filterwarnings("ignore")

In [2]:
conn = psycopg2.connect(
host = "shopify-merchant-dump.ccm9mnr5avgs.ap-south-1.rds.amazonaws.com",
  user = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c",
  password = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_password",
  database = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_db",
  port = '5432'
)

In [3]:
# query = """select o.id as order_id,
#             o.created_at::date as order_date,
#             o.customer_id as user_id,
#             (o.total_discounts * 100) / o.total_line_items_price as discount_percent,
#             oli.product_id as product_id,
#             oli.name as product_name,
#             o.total_price as order_amount,
#             CASE
#         WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
#          ELSE 'Delivered'
#      END as order_status,
#             oli.quantity as quantity,
#             p.product_type as product_category,
#             oli.price as sp
#             from orders o
#             inner join order_line_item oli on oli.order_id = o.id
#             inner join products p on oli.product_id=p.id"""

In [4]:
# query = """select o.id as order_id,
#             o.created_at::date as order_date,
#             o.customer_id as user_id,
#             o.total_line_items_price as total_price,
#             o.total_discounts / o.total_line_items_price as discount_percent,
#             oli.product_id as product_id,
#             oli.name as product_name,
#             o.total_price as order_amount,
#             CASE
#         WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
#          ELSE 'Delivered'
#      END as order_status,
#             oli.quantity as quantity,
#             p.product_type as product_category,
#             oli.price as sp
#             from orders o
#             inner join order_line_item oli on oli.order_id = o.id
#             inner join products p on oli.product_id=p.id"""

query1='''
    select o.id as order_id,
    o.created_at::date as order_date,
    o.customer_id as user_id,
    o.total_line_items_price as MRP,
    o.total_discounts / o.total_line_items_price as discount_percent,
    o.total_discounts as discount_amount,
    o.total_price as total_price

    from orders o
    WHERE
    cancelled_at is NULL;
'''

query2='''
    select oli.order_id as order_id,
    oli.product_id as product_id,
    oli.name as product_name,
    oli.quantity as quantity,
    oli.price as sp
    from order_line_item oli;
'''

query3='''
select osa.order_id as order_id,
osa.address1 as delivery_address
from  order_shipping_address osa'''

query4='''
select p.id as product_id,
p.product_type as product_category
from  products p
'''

o=pd.read_sql(query1,conn)
r=pd.read_sql(query2,conn)
a=pd.read_sql(query3,conn)
p=pd.read_sql(query4,conn)
product=pd.merge(r,p,on='product_id',how='left')
data=pd.merge(o,product,on='order_id',how='left')
data=pd.merge(data,a,on='order_id',how='left')

In [5]:
data = data.drop_duplicates()
data = data.dropna()

In [6]:
# data[data['order_date']<='2023/07/31']
# data[data['order_date']>='2023/01/01']

In [7]:
data["order_date"] = pd.to_datetime(data["order_date"])
data[data["product_id"] == np.nan]
data = data.dropna()
data['WeekCount']=data['order_date'].dt.strftime('%Y-w%U')
data=data.sort_values('order_date')
WeekList=data['WeekCount'].unique()

In [8]:
# data = data[data['order_status']=='Delivered']

In [9]:
WeekList = WeekList.tolist()
output_list = []

for i in range(len(WeekList) - 1):
    output_list.append(WeekList[i])
    current_week = int(WeekList[i].split('-w')[1])
    next_week = int(WeekList[i + 1].split('-w')[1])
    missing_weeks = next_week - current_week - 1
    for j in range(1, missing_weeks + 1):
        missing_week = current_week + j
        output_list.append(f'{WeekList[i][:5]}w{missing_week:02d}')

output_list.append(WeekList[-1])

In [10]:
orderuserlevel=data.groupby(['order_id','user_id']).agg(
          order_date=pd.NamedAgg(column='order_date', aggfunc='min'),

    ).reset_index()
orderuserlevel=orderuserlevel.sort_values('order_date')
orderuserlevel['TrxnRank'] = orderuserlevel.groupby('user_id')['order_date'].rank(method='first')
orderuserlevel


data=pd.merge(data,
         orderuserlevel[['order_id','TrxnRank']],
         on="order_id",
         how ='left')
data['New']=0
data['Repeat']=0
data.loc[data['TrxnRank']==1,'New']=1
data.loc[data['TrxnRank']>1,'Repeat']=1

#AOV Contribution
data['ordervalue']=data['quantity']*data['sp']
data['AOVContribution']=data['ordervalue']/data["order_id"].nunique()
data['DeltaDaysinOrders']=(data.groupby('user_id').order_date.shift() - data.order_date).dt.days.abs()
data['DeltaOrderValues']=(data.groupby('user_id').ordervalue.shift() - data.ordervalue)
data['discount_ratio'] = data['discount_amount']/data['mrp'] #change
data['new_ordervalue']= data['ordervalue']-(data['ordervalue']*data['discount_ratio'])# change

In [11]:
Weekly_Stats = {}
productsegstats_weekly = {}
user_cohort_labels = ['New Users', 'Churned Users', 'Recent & Loyal Users', 'Champion Users', 'Repeat Churned Users']
user_cohort_var_list = ['Users']#, 'Orders', 'Recency', 'AOVContribution', 'OrderValue','MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV'
product_cohort_labels=['Hero Products', 'High Value & Slow Moving Products', 'Low Value & Slow Moving Products', 'New Business Products', 'High Repeat Products']
product_cohort_var_list=['ProductCount'] #'Orders','Users','AverageSellingPrice','QuantityperCart','CartPenetration','NewUserRatio','NewOrders','ASPXQpC','AOVContribution','ProductCountMix','OrdersMix','UsersMix','AOVContribution(%)','OrderPerSKU'
final_stats = pd.DataFrame()
avg_delta_days = data["DeltaDaysinOrders"].mean()
lst = []

for x in output_list:
  x3 = data[data['WeekCount'] <= x]
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: np.min(np.sort(x)[-2:])),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          #Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          #Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()
    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['MaturityAge']=np.nan
    userweeklydata.loc[userweeklydata['Frequency']>1,"MaturityAge"]=(userweeklydata['Last_Date']-userweeklydata['First_Date']).dt.days.abs()
    #userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    #userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    #userweeklydata['value_score'] = userweeklydata['Order_Value'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
   # userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score']=0
    userweeklydata.loc[userweeklydata['MaturityAge'].notnull(),'m_score']= userweeklydata.loc[userweeklydata['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    #userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    #userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables_3(df):
        if ((df['Frequency']==1) & (df['r_score']>2)):
            return "New Users"
        elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
            return 'Churned Users'
        elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
            return 'Recent & Loyal Users'
        elif ((df['Frequency'] > 1) & (df['m_score'] >=2) & (df['Recency'] <= avg_delta_days)):
            return 'Champion Users'
        else:
            return 'Repeat Churned Users'
    userweeklydata['User_Cohort_Label'] = userweeklydata.apply(CohortLables_3, axis=1)
    x3=pd.merge(x3,
         userweeklydata[['user_id','User_Cohort_Label']],
         on="user_id",
         how ='left')

    totalcarts=x3['order_id'].nunique()
    product_level=x3.groupby(['product_id', 'product_name']).agg(
            Carts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
            Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
            ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
            NewOrders=pd.NamedAgg(column='New', aggfunc='sum'),
            RepeatOrders=pd.NamedAgg(column='Repeat', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
            AOVContribution=pd.NamedAgg(column='AOVContribution', aggfunc='sum'),
            Total_Order_Value=pd.NamedAgg(column='new_ordervalue', aggfunc='sum'),
            Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique')
    ).reset_index()
    data_new=x3[x3['TrxnRank']==1]
    xuser_count = data_new.groupby(["product_name"]).agg(
    x_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique'),
    x_user_list = pd.NamedAgg(column = "user_id", aggfunc = lambda x: [x.unique()])
    ).reset_index()
    data_repeat=x3[x3['TrxnRank']==2]
    repeat_users=data_repeat['user_id'].to_list()
    l=[]
    for i in range (len(xuser_count)):
        a=(xuser_count.iloc[i,2])[0]
        b=[x for x in a if x in repeat_users]
        l.append(len(b))

    xuser_count['repeat_pool']=l

    yuser_count = data_repeat.groupby('product_name').agg(
    y_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')
    ).reset_index()
    main_grouped_for_title = pd.merge(xuser_count, yuser_count, on='product_name', how='left')
    main_grouped_for_title['y_trxn_pool'].fillna(0, inplace=True)
    result = main_grouped_for_title.copy()

    result['Conversion_Rate_x_to_y'] = result['repeat_pool']/result['x_trxn_pool']
    result['Conversion_Rate_x_to_y'].fillna(0, inplace= True)
    conversion_result = result
    productlevel=pd.merge(product_level,conversion_result,on='product_name', how='left')


    productlevel['NewRepeatConversionRatio']=productlevel['Conversion_Rate_x_to_y']
    productlevel['NewRepeatConversionRatio']=productlevel['NewRepeatConversionRatio'].fillna(0)
    productlevel=productlevel.sort_values('NewRepeatConversionRatio')
    productlevel['CartPenetration']=productlevel['Carts']/totalcarts
    productlevel['QpC']=productlevel['Quantity']/productlevel['Carts']
    productlevel['ASPXQpC']=productlevel['ASP']*productlevel['QpC']

    CP_labels, NewRatio_labels,ASP_labels, QpC_labels =  range(1,5), range(1,5),range(1,5),range(1,5)
    productlevel['CP_score'] = productlevel['CartPenetration'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = CP_labels)).astype('int64')
    productlevel['NewRepeatConversion_score'] = productlevel['NewRepeatConversionRatio'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = NewRatio_labels))#.astype('int64')
    productlevel['ASP_score'] = productlevel['ASP'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = ASP_labels)).astype('int64')
    # productlevel['QpC_score'] = productlevel['QpC'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = QpC_labels)).astype('int64')
    # productlevel['SumScore'] = productlevel['CP_score'] + productlevel['NewRepeatConversion_score'] + productlevel['ASP_score'] + productlevel['QpC_score']
    def ProductCohortLables2(df):
        if ((df['CP_score']>2) and (df['NewRepeatConversion_score']>2)):
            return 'Hero Products'
        elif df['NewRepeatConversion_score'] >2:
            return 'High Repeat Products'
        elif (df['CP_score'] >2):
            return 'New Business Products'
        elif (df['ASP_score'] >3):
            return 'High Value & Slow Moving Products'
        elif (df['ASP_score'] <=3):
            return 'Low Value & Slow Moving Products'
        else:
            return 'Low Value & Slow Moving Products'
    productlevel['Product Cohort Lables'] = productlevel.apply(ProductCohortLables2, axis=1)

    x3=pd.merge(x3,
         productlevel[['product_name','Product Cohort Lables']],
         on="product_name",
         how ='left')


    User_Product_Level=x3.groupby(['User_Cohort_Label','Product Cohort Lables']).agg(
        Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),

    ).reset_index()

    Weekly_Stats[x] = User_Product_Level
    x4 = data[data['WeekCount']==x]
    x5 = pd.merge(x4,userweeklydata[['user_id','User_Cohort_Label']], on='user_id', how='left')
    x6 = pd.merge(x5,productlevel[['product_name','Product Cohort Lables']], on='product_name', how='left')
    # final_stats = final_stats.append(x6, ignore_index=True)
    final_stats = pd.concat([final_stats, x6], ignore_index=True)

  except:
    lst.append(x)

In [12]:
data = data[~(data['WeekCount'].isin(lst))]
output_list = [item for item in output_list if item not in lst]

In [13]:
avg_delta_days = data["DeltaDaysinOrders"].mean()
avg_delta_days

39.16018018018018

In [14]:
Weekly_Stats = {}
productsegstats_weekly = {}
user_cohort_labels = ['New Users', 'Churned Users', 'Recent & Loyal Users', 'Champion Users', 'Repeat Churned Users']
user_cohort_var_list = ['Users']#, 'Orders', 'Recency', 'AOVContribution', 'OrderValue','MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV'
product_cohort_labels=['Hero Products', 'High Value & Slow Moving Products', 'Low Value & Slow Moving Products', 'New Business Products', 'High Repeat Products']
product_cohort_var_list=['ProductCount'] #'Orders','Users','AverageSellingPrice','QuantityperCart','CartPenetration','NewUserRatio','NewOrders','ASPXQpC','AOVContribution','ProductCountMix','OrdersMix','UsersMix','AOVContribution(%)','OrderPerSKU'
final_stats = pd.DataFrame()
avg_delta_days = data["DeltaDaysinOrders"].mean()

for x in output_list:
  x3 = data[data['WeekCount'] <= x]
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: np.min(np.sort(x)[-2:])),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          #Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          #Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()
    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['MaturityAge']=np.nan
    userweeklydata.loc[userweeklydata['Frequency']>1,"MaturityAge"]=(userweeklydata['Last_Date']-userweeklydata['First_Date']).dt.days.abs()
    #userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    #userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    #userweeklydata['value_score'] = userweeklydata['Order_Value'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
   # userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score']=0
    userweeklydata.loc[userweeklydata['MaturityAge'].notnull(),'m_score']= userweeklydata.loc[userweeklydata['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    #userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    #userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables_3(df):
        if ((df['Frequency']==1) & (df['r_score']>2)):
            return "New Users"
        elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
            return 'Churned Users'
        elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
            return 'Recent & Loyal Users'
        elif ((df['Frequency'] > 1) & (df['m_score'] >=2) & (df['Recency'] <= avg_delta_days)):
            return 'Champion Users'
        else:
            return 'Repeat Churned Users'
    userweeklydata['User_Cohort_Label'] = userweeklydata.apply(CohortLables_3, axis=1)
    x3=pd.merge(x3,
         userweeklydata[['user_id','User_Cohort_Label']],
         on="user_id",
         how ='left')

    totalcarts=x3['order_id'].nunique()
    product_level=x3.groupby(['product_id', 'product_name']).agg(
            Carts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
            Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
            ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
            NewOrders=pd.NamedAgg(column='New', aggfunc='sum'),
            RepeatOrders=pd.NamedAgg(column='Repeat', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
            AOVContribution=pd.NamedAgg(column='AOVContribution', aggfunc='sum'),
            Total_Order_Value=pd.NamedAgg(column='new_ordervalue', aggfunc='sum'),
            Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique')
    ).reset_index()
    data_new=x3[x3['TrxnRank']==1]
    xuser_count = data_new.groupby(["product_name"]).agg(
    x_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique'),
    x_user_list = pd.NamedAgg(column = "user_id", aggfunc = lambda x: [x.unique()])
    ).reset_index()
    data_repeat=x3[x3['TrxnRank']==2]
    repeat_users=data_repeat['user_id'].to_list()
    l=[]
    for i in range (len(xuser_count)):
        a=(xuser_count.iloc[i,2])[0]
        b=[x for x in a if x in repeat_users]
        l.append(len(b))

    xuser_count['repeat_pool']=l

    yuser_count = data_repeat.groupby('product_name').agg(
    y_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')
    ).reset_index()
    main_grouped_for_title = pd.merge(xuser_count, yuser_count, on='product_name', how='left')
    main_grouped_for_title['y_trxn_pool'].fillna(0, inplace=True)
    result = main_grouped_for_title.copy()

    result['Conversion_Rate_x_to_y'] = result['repeat_pool']/result['x_trxn_pool']
    result['Conversion_Rate_x_to_y'].fillna(0, inplace= True)
    conversion_result = result
    productlevel=pd.merge(product_level,conversion_result,on='product_name', how='left')


    productlevel['NewRepeatConversionRatio']=productlevel['Conversion_Rate_x_to_y']
    productlevel['NewRepeatConversionRatio']=productlevel['NewRepeatConversionRatio'].fillna(0)
    productlevel=productlevel.sort_values('NewRepeatConversionRatio')
    productlevel['CartPenetration']=productlevel['Carts']/totalcarts
    productlevel['QpC']=productlevel['Quantity']/productlevel['Carts']
    productlevel['ASPXQpC']=productlevel['ASP']*productlevel['QpC']

    CP_labels, NewRatio_labels,ASP_labels, QpC_labels =  range(1,5), range(1,5),range(1,5),range(1,5)
    productlevel['CP_score'] = productlevel['CartPenetration'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = CP_labels)).astype('int64')
    productlevel['NewRepeatConversion_score'] = productlevel['NewRepeatConversionRatio'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = NewRatio_labels))#.astype('int64')
    productlevel['ASP_score'] = productlevel['ASP'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = ASP_labels)).astype('int64')
    # productlevel['QpC_score'] = productlevel['QpC'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = QpC_labels)).astype('int64')
    # productlevel['SumScore'] = productlevel['CP_score'] + productlevel['NewRepeatConversion_score'] + productlevel['ASP_score'] + productlevel['QpC_score']
    def ProductCohortLables2(df):
        if ((df['CP_score']>2) and (df['NewRepeatConversion_score']>2)):
            return 'Hero Products'
        elif df['NewRepeatConversion_score'] >2:
            return 'High Repeat Products'
        elif (df['CP_score'] >2):
            return 'New Business Products'
        elif (df['ASP_score'] >3):
            return 'High Value & Slow Moving Products'
        elif (df['ASP_score'] <=3):
            return 'Low Value & Slow Moving Products'
        else:
            return 'Low Value & Slow Moving Products'
    productlevel['Product Cohort Lables'] = productlevel.apply(ProductCohortLables2, axis=1)

    x3=pd.merge(x3,
         productlevel[['product_name','Product Cohort Lables']],
         on="product_name",
         how ='left')


    User_Product_Level=x3.groupby(['User_Cohort_Label','Product Cohort Lables']).agg(
        Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),

    ).reset_index()

    Weekly_Stats[x] = User_Product_Level
    x4 = data[data['WeekCount']==x]
    x5 = pd.merge(x4,userweeklydata[['user_id','User_Cohort_Label']], on='user_id', how='left')
    x6 = pd.merge(x5,productlevel[['product_name','Product Cohort Lables']], on='product_name', how='left')
    # final_stats = final_stats.append(x6, ignore_index=True)
    final_stats = pd.concat([final_stats, x6], ignore_index=True)

  except:
    pass

In [15]:
final_stats

,order_id,order_date,user_id,mrp,discount_percent,discount_amount,total_price,product_id,product_name,quantity,...,New,Repeat,ordervalue,AOVContribution,DeltaDaysinOrders,DeltaOrderValues,discount_ratio,new_ordervalue,User_Cohort_Label,Product Cohort Lables
0,2424016666762,2020-09-20,3.563086e+12,650.0,0.300000,195.0,455.0,5.321724e+12,Easy Brew - Cold Brew Bags (Pack of 4),1,...,1,0,650.0,0.010281,NaN,NaN,0.300000,455.000000,Churned Users,Hero Products
1,2424016142474,2020-09-20,3.563085e+12,650.0,0.300000,195.0,455.0,5.321724e+12,Easy Brew - Cold Brew Bags (Pack of 4),1,...,1,0,650.0,0.010281,NaN,NaN,0.300000,455.000000,Churned Users,Hero Products
2,2423709794442,2020-09-20,3.562684e+12,650.0,0.300000,195.0,455.0,5.321724e+12,Easy Brew - Cold Brew Bags (Pack of 4),1,...,1,0,650.0,0.010281,NaN,NaN,0.300000,455.000000,Repeat Churned Users,Hero Products
3,2423748001930,2020-09-20,3.562720e+12,650.0,0.300000,195.0,455.0,5.321724e+12,Easy Brew - Cold Brew Bags (Pack of 4),1,...,1,0,650.0,0.010281,NaN,NaN,0.300000,455.000000,Churned Users,Hero Products
4,2424011423882,2020-09-20,3.563072e+12,650.0,0.300000,195.0,455.0,5.321724e+12,Easy Brew - Cold Brew Bags (Pack of 4),1,...,1,0,650.0,0.010281,NaN,NaN,0.300000,455.000000,Churned Users,Hero Products
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83616,5382558941322,2024-01-07,5.073752e+12,1100.0,0.400000,440.0,660.0,5.320286e+12,Baba Budangiri Estate Coffee - Whole Beans / 5...,1,...,0,1,1100.0,0.017398,258.0,-330.0,0.400000,660.000000,Repeat Churned Users,Hero Products
83617,5383258538122,2024-01-08,6.658749e+12,1260.0,0.500000,630.0,630.0,5.321622e+12,Peaberry Coffee - Medium Roast - French Press ...,1,...,0,1,630.0,0.009965,97.0,0.0,0.500000,315.000000,Champion Users,Hero Products
83618,5383258538122,2024-01-08,6.658749e+12,1260.0,0.500000,630.0,630.0,6.624082e+12,Blueberry Mocha - Speciality Blend - 250 gm / ...,1,...,0,1,630.0,0.009965,0.0,0.0,0.500000,315.000000,Champion Users,Hero Products
83619,5383244251274,2024-01-08,6.339903e+12,1135.0,0.484581,550.0,585.0,5.320258e+12,Thippanahalli Estate Coffee - French Press / 2...,1,...,0,1,585.0,0.009253,89.0,0.0,0.484581,301.519824,Champion Users,Hero Products


In [16]:
last_tag_df = userweeklydata[["user_id" , "User_Cohort_Label"]]

In [17]:
def group_consecutive(lst):
    result = []
    current_group = []

    for key, group in groupby(lst):
        key = next(group)
        if current_group and current_group[-1] == key:
            continue

        current_group.append(key)

    result.extend(current_group)
    return result

customer_profile_df = final_stats.groupby("user_id").agg(
    Order_First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
    Order_Last_Date=pd.NamedAgg(column='order_date', aggfunc='max'),
    Total_Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
    Total_Order_Counts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
    User_Journey=pd.NamedAgg(column='User_Cohort_Label', aggfunc=lambda x: group_consecutive(x.tolist())),
    User_First_Tag=pd.NamedAgg(column='User_Cohort_Label', aggfunc=lambda x: group_consecutive(x.tolist())[0]),
    Total_Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
    Mean_Delta_Days_in_Orders=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
    Mean_Delta_Order_Values=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean'),
).reset_index()
globalmaxdateweek=final_stats['order_date'].max()
customer_profile_df["AOV"] = customer_profile_df["Total_Order_Value"]/customer_profile_df["Total_Order_Counts"]
customer_profile_df["ASP"] = customer_profile_df["Total_Order_Value"]/customer_profile_df["Total_Quantity"]
customer_profile_df['Recency'] = (globalmaxdateweek - customer_profile_df['Order_Last_Date']).dt.days.astype(int)
customer_profile_df['Maturity']=np.nan
customer_profile_df.loc[customer_profile_df['Total_Order_Counts']>1,"Maturity"]=(customer_profile_df['Order_Last_Date']-customer_profile_df['Order_First_Date']).dt.days.abs()
customer_profile_df['Mean_Delta_Days_in_Orders'] = np.where(customer_profile_df['Total_Order_Counts'] == 1,np.nan,customer_profile_df['Mean_Delta_Days_in_Orders'])
customer_profile_df['Mean_Delta_Order_Values'] = np.where(customer_profile_df['Total_Order_Counts'] == 1,np.nan,customer_profile_df['Mean_Delta_Order_Values'])

In [18]:
kbc=pd.merge(customer_profile_df,last_tag_df, on ="user_id", how="left")

In [19]:
kbc.rename(columns={"User_Cohort_Label": "User_Final_Tag"}, inplace=True)

In [20]:
customer_prdt = data.groupby("user_id").agg(
    Product_Names_List = pd.NamedAgg(column='product_name', aggfunc=lambda x: list(pd.unique(x))),
    Product_Category_List = pd.NamedAgg(column='product_category', aggfunc=lambda x: list(pd.unique(x))),
    Average_Discount_Percentage = pd.NamedAgg(column='discount_percent', aggfunc="mean"),
    Average_Discount_List = pd.NamedAgg(column='discount_percent', aggfunc=lambda x: list(pd.unique(x))),
    AOV_List = pd.NamedAgg(column='total_price', aggfunc=lambda x: list(pd.unique(x))),
).reset_index()

In [21]:
final_df_new_uplaod = pd.merge(kbc, customer_prdt, on="user_id", how="left")

In [22]:
final_df_median_value = final_df_new_uplaod['Average_Discount_Percentage'].median()
final_df_new_uplaod['Discount_Category'] = pd.cut(final_df_new_uplaod['Average_Discount_Percentage'],
                                                 bins=[float('-inf'), 0, final_df_median_value, float('inf')],
                                                 labels=['No Discount', 'Lower Discount', 'Higher Discount'])

In [23]:
query1 = '''select
    created_at::date as OrderDate,
    id as order_id,
    customer_id as user_id,
    CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
        ELSE 'Delivered'
    END as order_status
    from orders;'''

query2 = '''select
    name as product_name,
    order_id,
    product_id,
    quantity,
    price,
    variant_title as variant
    from order_line_item;'''

query3 = '''select
    id as product_id,
    title as product_title,
    product_type as product_category,
    handle,
    tags
    from products;'''

In [24]:
e = pd.read_sql(query1, conn)
f = pd.read_sql(query2, conn)
g = pd.read_sql(query3, conn)

In [25]:
# product_name, product_id, variant, product_title, product_category, handle, tags
parameter_input = "product_category"

In [26]:
e = e[e['order_status']=='Delivered']
e.drop('order_status',axis=1,inplace=True)

In [27]:
e_sorted = e.sort_values(by=['user_id', 'orderdate', 'order_id'])
# e['Transaction_Rank'] = e.groupby(['user_id'])['orderdate'].rank(method='first')

In [28]:
merged_df = pd.merge(f, e, on='order_id')
merged_df.dropna(inplace = True)

In [29]:
final = pd.merge(merged_df, g, on='product_id')
final.dropna(inplace = True)
final['Transaction_Rank'] = final.groupby(['user_id',parameter_input])['orderdate'].rank(method='first')

In [30]:
final['Total Order Value'] = final['quantity'] * final['price']

In [31]:
def main(final, transactionx, transactiony, parameter):

    final1 = final.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    if parameter == "tags":
        final_tags = final1.copy()
        final_tags['tags'] = final_tags['tags'].str.split(',')
        final_tags = final_tags.explode('tags', ignore_index=True)
        final1 = final_tags
    else:
        pass

    #-------------------------------------------------------------------------------------------------------------------------------

    trans_x = final1[final1['Transaction_Rank']==transactionx]
    trans_y = final1[final1['Transaction_Rank']==transactiony]

    #-------------------------------------------------------------------------------------------------------------------------------

    xuser_count = trans_x.groupby(parameter).agg(
        x_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')).reset_index()

    yuser_count = trans_y.groupby(parameter).agg(
        y_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')).reset_index()


    #-------------------------------------------------------------------------------------------------------------------------------

    main_grouped_for_title = pd.merge(xuser_count, yuser_count, on=parameter, how='left')

    #-------------------------------------------------------------------------------------------------------------------------------

    main_grouped_for_title['y_trxn_pool'].fillna(0, inplace=True)

    #-------------------------------------------------------------------------------------------------------------------------------

    result = main_grouped_for_title.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    result['Conversion_Rate_x_to_y'] = result['y_trxn_pool']/result['x_trxn_pool']
    result['Conversion_Rate_x_to_y'].fillna(0, inplace= True)

    #-------------------------------------------------------------------------------------------------------------------------------

    result['Cart_Penetration_for_x_transaction'] = result['x_trxn_pool']/(result['x_trxn_pool'].sum())
    result['Cart_Penetration_for_y_transaction'] = result['y_trxn_pool']/(result['y_trxn_pool'].sum())

    #-------------------------------------------------------------------------------------------------------------------------------

    result1 = result.copy()
    result2 = result.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    result1 = result1.sort_values(by='Conversion_Rate_x_to_y', ascending=False)
    result2 = result2.sort_values(by='Cart_Penetration_for_x_transaction', ascending=False)

    #-------------------------------------------------------------------------------------------------------------------------------

    result1['Mix']= result1['Conversion_Rate_x_to_y']*100/result1['Conversion_Rate_x_to_y'].sum()
    result1['CumuSum']= result1['Mix'].cumsum()
    result1['Conversion Rate Bin']=""
    result1.loc[result1['CumuSum']<=75,"Conversion Rate Bin"]='HighConversion'
    result1.loc[(result1['CumuSum']>75) & (result1['CumuSum']<=101),"Conversion Rate Bin"]='LowConversion'

    result2['Mix']= result2['Cart_Penetration_for_x_transaction']*100/result2['Cart_Penetration_for_x_transaction'].sum()
    result2['CumuSum']= result2['Mix'].cumsum()
    result2['Cart Penetration Bin']=""
    result2.loc[result2['CumuSum']<=75,"Cart Penetration Bin"]='HighCartPenetration'
    result2.loc[(result2['CumuSum']>75) & (result2['CumuSum']<=101),"Cart Penetration Bin"]='LowCartPenetration'

    #-------------------------------------------------------------------------------------------------------------------------------

    result2 = result2[[parameter,'Cart Penetration Bin']]
    final_merged_on_title_x_to_y = pd.merge(result1, result2, on=parameter)

    #-------------------------------------------------------------------------------------------------------------------------------

    final_merged_on_title_x_to_y = final_merged_on_title_x_to_y[final_merged_on_title_x_to_y['Conversion Rate Bin'] != 0]

    #-------------------------------------------------------------------------------------------------------------------------------

    final_merged_on_title_x_to_y['Bin'] = final_merged_on_title_x_to_y['Conversion Rate Bin'] + '_' + final_merged_on_title_x_to_y['Cart Penetration Bin']
    # final_merged_on_title_x_to_y.drop(['Conversion Rate Bin','Cart Penetration Bin'], axis=1, inplace= True)

    #-------------------------------------------------------------------------------------------------------------------------------

    merged_title_on_bin = final_merged_on_title_x_to_y.groupby('Bin').agg({
        parameter: 'nunique'
    }).reset_index()

    #-------------------------------------------------------------------------------------------------------------------------------

    x = final_merged_on_title_x_to_y.columns
    y = []
    for i in x:
        if "x_" in i:
            i = i.replace("x_", f"{str(transactionx)}_")
        if "_x" in i:
            i = i.replace("_x", f"_{str(transactiony)}")
        if "y_" in i:
            i = i.replace("y_", f"{str(transactiony)}_")
        if "_y" in i:
            i = i.replace("_y", f"_{str(transactiony)}")
        y.append(i)
    final_merged_on_title_x_to_y.columns = y

    #-------------------------------------------------------------------------------------------------------------------------------

    return final_merged_on_title_x_to_y, final_merged_on_title_x_to_y[[parameter, f"Conversion_Rate_{transactionx}_to_{transactiony}", f"Cart_Penetration_for_{transactionx}_transaction", f"Cart_Penetration_for_{transactiony}_transaction"]]

In [32]:
prdt_page, input = main(final, 1, 2, parameter_input)

In [33]:
def component1(final, transactionx, transactiony, parameter):
    prdt_page, input = main(final, transactionx, transactiony, parameter)
    component1_dump = pd.merge(final, prdt_page, on=parameter_input, how='left')
    acq = f"{transactionx}_trxn_pool"
    repeat = f"{transactiony}_trxn_pool"
    conversion = f"Conversion_Rate_{transactionx}_to_{transactiony}"
    cart_pen_1 = f"Cart_Penetration_for_{transactionx}_transaction"
    cart_pen_2 = f"Cart_Penetration_for_{transactiony}_transaction"
    component1_to_show = component1_dump.groupby("Bin").agg(
        Product_Counts = pd.NamedAgg(column = 'product_name', aggfunc = 'nunique'),
        Quantity = pd.NamedAgg(column = 'quantity', aggfunc = 'sum'),
        Acquired_Pool = pd.NamedAgg(column = acq, aggfunc = 'sum'),
        Repeat_Pool = pd.NamedAgg(column = repeat, aggfunc = 'sum'),
        Total_Order_Value = pd.NamedAgg(column = "Total Order Value", aggfunc = 'sum'),
        rename1 = pd.NamedAgg(column = conversion, aggfunc = 'mean'),
        rename2 = pd.NamedAgg(column = cart_pen_1, aggfunc = 'mean'),
        rename3 = pd.NamedAgg(column = cart_pen_2, aggfunc = 'mean')).reset_index()
    component1_to_show.rename(columns = {"rename1": conversion, "rename2": cart_pen_1, "rename3":cart_pen_2}, inplace = True)
    component1_to_show.set_index("Bin", inplace = True)
    return component1_to_show, component1_dump

In [34]:
component_1_to_show, dump = component1(final, 1, 2, parameter_input)

In [35]:
final_df_new_uplaod["Suggested_Assortment"] = ""
for i in range (len(final_df_new_uplaod)):
    already_purchased = len(final_df_new_uplaod.iloc[i, 15])
    if already_purchased < 10:
        categories = final_df_new_uplaod.iloc[i, 16]
        analyse = dump[dump["product_category"].isin(categories)]
        analyse = analyse.sort_values(by = 'Conversion_Rate_1_to_2', ascending = False)
        num = 10 - already_purchased
        sub_assortment = analyse.head(num)["product_name"].unique()
        assortment = sub_assortment
        final_df_new_uplaod.at[i, "Suggested_Assortment"] = [assortment]
    else:
        final_df_new_uplaod.at[i, "Suggested_Assortment"] = (final_df_new_uplaod.iloc[i, 15])[:10]

In [36]:
final_df_new_uplaod = final_df_new_uplaod[['user_id',
 'Order_First_Date',
 'Order_Last_Date',
 'Total_Order_Value',
 'Total_Order_Counts',
 'User_Journey',
 'User_First_Tag',
 'User_Final_Tag',
 'Total_Quantity',
 'Mean_Delta_Days_in_Orders',
 'Mean_Delta_Order_Values',
 'AOV',
 'ASP',
 'Recency',
 'Maturity',
 'Product_Names_List',
 'Product_Category_List',
 'Average_Discount_Percentage',
 'Average_Discount_List',
 'AOV_List',
 'Discount_Category',
 'Suggested_Assortment']]

In [37]:
final_df_new_uplaod['Order_Count_Without_Discount'] = final_df_new_uplaod['Average_Discount_List'].apply(
    lambda discount_list: sum(1 for value in discount_list if isinstance(value, (int, float)) and value == 0.0)
)

final_df_new_uplaod['Order_Count_With_Discount'] = final_df_new_uplaod['Average_Discount_List'].apply(
    lambda discount_list: sum(1 for value in discount_list if isinstance(value, (int, float)) and value > 0.0)
)


In [38]:
# final_df_new_uplaod['Order_Count_Without_Discount'] = 0
# final_df_new_uplaod['Order_Count_With_Discount'] = 0
# def count_positive_discounts(discount_list):
#     return len([value for value in discount_list if isinstance(value, (int, float)) and value > 0.0])

# def count_zero_discounts(discount_list):
#     return len([value for value in discount_list if isinstance(value, (int, float)) and value == 0.0])
# for index, row in final_df_new_uplaod.iterrows():
#     discount_list = row['Average_Discount_List']
#     final_df_new_uplaod['Order_Count_With_Discount'] = final_df_new_uplaod['Average_Discount_List'].apply(count_positive_discounts)
#     final_df_new_uplaod['Order_Count_Without_Discount'] = final_df_new_uplaod['Average_Discount_List'].apply(count_zero_discounts)

In [39]:
final_df_new_uplaod["No_of_SKUs"] = final_df_new_uplaod['Product_Names_List'].apply(len)

In [40]:
final_df_new_uplaod.to_csv("Customer_Profile_for_TCR_new_new.csv", index = False)

In [41]:
def component_1(final_df_new_upload):
    df = pd.DataFrame()

    df['user_id'] = final_df_new_upload["user_id"]
    df['Order_First_Date'] = final_df_new_upload['Order_First_Date']
    df['Order_Last_Date'] = final_df_new_upload['Order_Last_Date']
    df['No_of_Orders'] = final_df_new_upload["Total_Order_Counts"]
    df["Tag"] = final_df_new_upload["User_Final_Tag"]
    df["AOV"] = final_df_new_upload["AOV"]
    df["Recency"] = final_df_new_upload["Recency"]
    df["Discount_Percentage"] = final_df_new_upload["Average_Discount_Percentage"]
    df["No_of_SKUs"] = final_df_new_uplaod['No_of_SKUs']

    return df

result = component_1(final_df_new_uplaod)

In [42]:
result

,user_id,Order_First_Date,Order_Last_Date,No_of_Orders,Tag,AOV,Recency,Discount_Percentage,No_of_SKUs
0,2.912877e+12,2021-07-09,2023-12-01,4,Repeat Churned Users,576.000000,38,0.285000,3
1,3.554806e+12,2020-10-15,2020-12-02,35,Repeat Churned Users,739.428571,1132,0.996478,33
2,3.555623e+12,2023-01-11,2023-01-11,1,Churned Users,2100.000000,362,0.600000,1
3,3.555652e+12,2022-10-23,2022-10-23,1,Churned Users,1000.000000,442,0.275000,2
4,3.555746e+12,2020-10-10,2022-10-28,15,Repeat Churned Users,2123.866667,437,0.613038,7
...,...,...,...,...,...,...,...,...,...
44774,6.987795e+12,2024-01-06,2024-01-06,1,New Users,950.000000,2,0.400000,1
44775,6.988563e+12,2024-01-07,2024-01-07,1,New Users,585.000000,1,0.400000,1
44776,6.988957e+12,2024-01-07,2024-01-07,1,New Users,199.000000,1,0.000000,1
44777,6.989049e+12,2024-01-07,2024-01-07,1,New Users,1200.000000,1,0.400000,1


In [43]:
final_df_new_uplaod

,user_id,Order_First_Date,Order_Last_Date,Total_Order_Value,Total_Order_Counts,User_Journey,User_First_Tag,User_Final_Tag,Total_Quantity,Mean_Delta_Days_in_Orders,...,Product_Names_List,Product_Category_List,Average_Discount_Percentage,Average_Discount_List,AOV_List,Discount_Category,Suggested_Assortment,Order_Count_Without_Discount,Order_Count_With_Discount,No_of_SKUs
0,2.912877e+12,2021-07-09,2023-12-01,2304.0,4,"[New Users, Repeat Churned Users]",New Users,Repeat Churned Users,5,291.666667,...,[Thogarihunkal Estate Coffee - French Press / ...,"[Single Origin Coffee, Coffee]",0.285000,"[0.24, 0.5, 0.4, 0.0]","[444.6, 490.0, 294.0, 274.0]",Higher Discount,[[Attikan Estate Coffee - 250 gm / Whole Beans...,1,3,3
1,3.554806e+12,2020-10-15,2020-12-02,25880.0,35,"[Recent & Loyal Users, Champion Users]",Recent & Loyal Users,Repeat Churned Users,35,2.257143,...,[Baba Budangiri Estate Coffee - Whole Beans / ...,"[Speciality Coffee Series, Small Batch Series,...",0.996478,"[0.9981818181818182, 0.9993746091307066, 0.980...","[1.0, 12.0, 11.0]",Higher Discount,[Baba Budangiri Estate Coffee - Whole Beans / ...,0,8,33
2,3.555623e+12,2023-01-11,2023-01-11,2100.0,1,[New Users],New Users,Churned Users,1,NaN,...,[Brazil Cerrado Coffee (Latin America) - Whole...,[],0.600000,[0.6],[840.0],Higher Discount,[[Blueberry Mocha - Speciality Blend - 250 gm ...,0,1,1
3,3.555652e+12,2022-10-23,2022-10-23,1000.0,1,[New Users],New Users,Churned Users,2,NaN,...,[Mullayangiri Estate Coffee - 250 gm / French ...,"[Single Origin Coffee, Coffee Accessories]",0.275000,[0.275],[725.0],Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,2
4,3.555746e+12,2020-10-10,2022-10-28,31858.0,15,"[New Users, Champion Users, Repeat Churned Use...",New Users,Repeat Churned Users,20,53.428571,...,[Thippanahalli Estate Coffee - French Press / ...,"[Single Origin Coffee, Speciality Coffee Serie...",0.613038,"[0.0, 0.9981818181818182, 1.0, 0.2101105845181...","[585.0, 1.0, 440.0, 0.0, 9000.0, 1050.0, 12999.0]",Higher Discount,[[Thippanahalli Estate Coffee - Whole Beans / ...,1,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44774,6.987795e+12,2024-01-06,2024-01-06,950.0,1,[New Users],New Users,New Users,1,NaN,...,[Easy Brew (Hot Brew) - Whiskey Barrel Aged Co...,[Easy Brew],0.400000,[0.4],[570.0],Higher Discount,[[Easy Brew (Hot Brew) - Mysore Nuggets - Pack...,0,1,1
44775,6.988563e+12,2024-01-07,2024-01-07,585.0,1,[New Users],New Users,New Users,1,NaN,...,[Thippanahalli Estate Coffee - French Press / ...,[Single Origin Coffee],0.400000,[0.4],[351.0],Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,1
44776,6.988957e+12,2024-01-07,2024-01-07,199.0,1,[New Users],New Users,New Users,1,NaN,...,[South Indian Traditional Filter Coffee - Dark...,[South Indian Filter Coffee],0.000000,[0.0],[199.0],No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1
44777,6.989049e+12,2024-01-07,2024-01-07,1200.0,1,[New Users],New Users,New Users,1,NaN,...,[Single Malt Whiskey Barrel Aged Coffee - Fren...,[Small Batch Series],0.400000,[0.4],[720.0],Higher Discount,[[Single Malt Whiskey Barrel Aged Coffee - Hom...,0,1,1


In [44]:
final_df_median_value_aov = final_df_new_uplaod['AOV'].median()
final_df_new_uplaod['AOV_Category'] = pd.cut(final_df_new_uplaod['AOV'],
                                                 bins=[float('-inf'), final_df_median_value_aov, float('inf')],
                                                 labels=['Lower AOV', 'Higher AOV'])

In [45]:
final_df_median_value_order_count = final_df_new_uplaod['Total_Order_Counts'].median()
final_df_new_uplaod['Order_Count_Category'] = pd.cut(final_df_new_uplaod['Total_Order_Counts'],
                                                 bins=[float('-inf'), final_df_median_value_order_count, float('inf')],
                                                 labels=['Lower Orders', 'Higher Orders'])

In [46]:
final_df_new_uplaod

,user_id,Order_First_Date,Order_Last_Date,Total_Order_Value,Total_Order_Counts,User_Journey,User_First_Tag,User_Final_Tag,Total_Quantity,Mean_Delta_Days_in_Orders,...,Average_Discount_Percentage,Average_Discount_List,AOV_List,Discount_Category,Suggested_Assortment,Order_Count_Without_Discount,Order_Count_With_Discount,No_of_SKUs,AOV_Category,Order_Count_Category
0,2.912877e+12,2021-07-09,2023-12-01,2304.0,4,"[New Users, Repeat Churned Users]",New Users,Repeat Churned Users,5,291.666667,...,0.285000,"[0.24, 0.5, 0.4, 0.0]","[444.6, 490.0, 294.0, 274.0]",Higher Discount,[[Attikan Estate Coffee - 250 gm / Whole Beans...,1,3,3,Higher AOV,Higher Orders
1,3.554806e+12,2020-10-15,2020-12-02,25880.0,35,"[Recent & Loyal Users, Champion Users]",Recent & Loyal Users,Repeat Churned Users,35,2.257143,...,0.996478,"[0.9981818181818182, 0.9993746091307066, 0.980...","[1.0, 12.0, 11.0]",Higher Discount,[Baba Budangiri Estate Coffee - Whole Beans / ...,0,8,33,Higher AOV,Higher Orders
2,3.555623e+12,2023-01-11,2023-01-11,2100.0,1,[New Users],New Users,Churned Users,1,NaN,...,0.600000,[0.6],[840.0],Higher Discount,[[Blueberry Mocha - Speciality Blend - 250 gm ...,0,1,1,Higher AOV,Lower Orders
3,3.555652e+12,2022-10-23,2022-10-23,1000.0,1,[New Users],New Users,Churned Users,2,NaN,...,0.275000,[0.275],[725.0],Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,2,Higher AOV,Lower Orders
4,3.555746e+12,2020-10-10,2022-10-28,31858.0,15,"[New Users, Champion Users, Repeat Churned Use...",New Users,Repeat Churned Users,20,53.428571,...,0.613038,"[0.0, 0.9981818181818182, 1.0, 0.2101105845181...","[585.0, 1.0, 440.0, 0.0, 9000.0, 1050.0, 12999.0]",Higher Discount,[[Thippanahalli Estate Coffee - Whole Beans / ...,1,3,7,Higher AOV,Higher Orders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44774,6.987795e+12,2024-01-06,2024-01-06,950.0,1,[New Users],New Users,New Users,1,NaN,...,0.400000,[0.4],[570.0],Higher Discount,[[Easy Brew (Hot Brew) - Mysore Nuggets - Pack...,0,1,1,Higher AOV,Lower Orders
44775,6.988563e+12,2024-01-07,2024-01-07,585.0,1,[New Users],New Users,New Users,1,NaN,...,0.400000,[0.4],[351.0],Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,1,Higher AOV,Lower Orders
44776,6.988957e+12,2024-01-07,2024-01-07,199.0,1,[New Users],New Users,New Users,1,NaN,...,0.000000,[0.0],[199.0],No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders
44777,6.989049e+12,2024-01-07,2024-01-07,1200.0,1,[New Users],New Users,New Users,1,NaN,...,0.400000,[0.4],[720.0],Higher Discount,[[Single Malt Whiskey Barrel Aged Coffee - Hom...,0,1,1,Higher AOV,Lower Orders


In [47]:
def component_2(final_df_new_upload, level):
    df = pd.DataFrame()
    first_part, second_part = level.split(" ")
    if second_part == "AOV":
      filtered_df = final_df_new_uplaod[final_df_new_uplaod["AOV_Category"] == level]
    if second_part == "Orders":
      filtered_df = final_df_new_uplaod[final_df_new_uplaod["Order_Count_Category"] == level]
    if second_part == "Discount":
      filtered_df = final_df_new_uplaod[final_df_new_uplaod["Discount_Category"] == level]


    df['user_id'] = filtered_df["user_id"]
    df['Order_First_Date'] = filtered_df['Order_First_Date']
    df['Order_Last_Date'] = filtered_df['Order_Last_Date']
    df['No_of_Orders'] = filtered_df["Total_Order_Counts"]
    df["Tag"] = filtered_df["User_Final_Tag"]
    df["AOV"] = filtered_df["AOV"]
    df["Recency"] = filtered_df["Recency"]
    df["Discount_Percentage"] = filtered_df["Average_Discount_Percentage"]
    df["No_of_SKUs"] = filtered_df['No_of_SKUs']

    return df

result = component_2(final_df_new_uplaod, "Lower AOV")

In [48]:
result

,user_id,Order_First_Date,Order_Last_Date,No_of_Orders,Tag,AOV,Recency,Discount_Percentage,No_of_SKUs
481,3.568465e+12,2020-09-24,2022-12-25,3,Repeat Churned Users,543.333333,379,0.648718,3
669,3.570138e+12,2020-11-23,2020-11-23,1,Churned Users,550.000000,1141,0.000018,1
677,3.571057e+12,2023-07-05,2023-07-05,1,New Users,490.000000,187,0.300000,1
738,3.575041e+12,2020-09-30,2020-09-30,1,Churned Users,550.000000,1195,0.000000,1
739,3.577734e+12,2020-10-02,2020-10-23,2,Repeat Churned Users,550.000000,1172,0.000000,2
...,...,...,...,...,...,...,...,...,...
44770,6.987524e+12,2024-01-06,2024-01-06,1,New Users,199.000000,2,0.000000,1
44771,6.987531e+12,2024-01-06,2024-01-06,1,New Users,199.000000,2,0.000000,1
44772,6.987560e+12,2024-01-06,2024-01-06,1,New Users,199.000000,2,0.000000,1
44773,6.987786e+12,2024-01-06,2024-01-06,1,New Users,199.000000,2,0.000000,1


In [80]:
query1='''
    select o.id as order_id,
    o.created_at::date as order_date,
    o.customer_id as user_id,
    o.total_line_items_price as MRP,
    o.total_discounts / o.total_line_items_price as discount_percent,
    o.total_discounts as discount_amount,
    o.total_price as total_price

    from orders o
    WHERE
    cancelled_at is NULL;
'''

query2='''
    select oli.order_id as order_id,
    oli.product_id as product_id,
    oli.name as product_name,
    oli.quantity as quantity,
    oli.price as sp
    from order_line_item oli;
'''

query3='''
select osa.order_id as order_id,
osa.address1 as delivery_address
from  order_shipping_address osa'''

query4='''
select p.id as product_id,
p.product_type as product_category
from  products p
'''

o=pd.read_sql(query1,conn)
r=pd.read_sql(query2,conn)
a=pd.read_sql(query3,conn)
p=pd.read_sql(query4,conn)
product=pd.merge(r,p,on='product_id',how='left')
data=pd.merge(o,product,on='order_id',how='left')
data1=pd.merge(data,a,on='order_id',how='left')

In [82]:
data1 = data1.drop_duplicates()
data1 = data1.dropna()

In [83]:
# import pandas as pd
# import re

# def clean_and_compare_addresses(customer_address, delivery_address):
#     # Remove punctuation and split into individual words
#     customer_words = set(re.findall(r'\b\w+\b', customer_address.lower()))
#     delivery_words = set(re.findall(r'\b\w+\b', delivery_address.lower()))

#     # Check if there are no words in customer_address
#     if not customer_words:
#         return "Gifting"

#     # Calculate the percentage of common words
#     common_words_percentage = len(customer_words.intersection(delivery_words)) / len(customer_words) * 100

#     # Check if at least 70% of words in customer_address are present in delivery_address
#     if common_words_percentage >= 70:
#         result = "Personal Consumption"
#     else:
#         result = "Gifting"

#     return result

# # Apply the function to each row
# filtered_df['result'] = filtered_df.apply(lambda row: clean_and_compare_addresses(row['customer_address'], row['delivery_address']), axis=1)

# # Calculate common_words_percentage for each row
# filtered_df['common_words_percentage'] = filtered_df.apply(lambda row: len(set(re.findall(r'\b\w+\b', row['customer_address'].lower())).intersection(set(re.findall(r'\b\w+\b', row['delivery_address'].lower())))) / len(set(re.findall(r'\b\w+\b', row['customer_address'].lower()))) * 100 if len(set(re.findall(r'\b\w+\b', row['customer_address'].lower()))) > 0 else 0, axis=1)

# # Display the DataFrame with the new column
# print(filtered_df)


In [86]:
filtered_df = data1.copy()

In [87]:
customer_address = filtered_df.groupby(["user_id"]).agg(
    Delivery_Address_Counts=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: x.value_counts().reset_index()),
).reset_index()

In [88]:
customer_address["Delivery_Address_Counts"].iloc[20]

array([['203/ A-wing, Bldg no.8, near RS Mani supermarket, Tilak Nagar, chembur',
        1]], dtype=object)

In [89]:
customer_address["user_id"].iloc[0]

2912876953738.0

In [90]:
result_rows = []

for i in range(len(customer_address)):
    user_id = customer_address.at[i, 'user_id']
    data = customer_address.at[i, 'Delivery_Address_Counts']

    # Find the index of the maximum count
    max_index = max(range(len(data)), key=lambda x: data[x][1])

    # Add the primary address to the result DataFrame
    result_rows.append({'user_id': user_id,
                        'address_type': 'primary',
                        'delivery_address': data[max_index][0],
                        'count': data[max_index][1]})

    # Add the rest of the addresses as gifting addresses to the result DataFrame
    result_rows.extend({'user_id': user_id,
                        'address_type': 'gifting',
                        'delivery_address': element,
                        'count': count} for j, (element, count) in enumerate(data) if j != max_index)

# Create the result DataFrame from the list of rows
result_df = pd.DataFrame(result_rows, columns=['user_id', 'address_type', 'delivery_address', 'count'])


In [91]:
result_df

,user_id,address_type,delivery_address,count
0,2.912877e+12,primary,LODHA Aurum Grande. Kanjurmarg East,2
1,2.912877e+12,gifting,Electronics City Phase 1 Electronic City,1
2,2.912877e+12,gifting,"269/127 Birhana, Devi dayal Marg , Lucknow",1
3,3.554806e+12,primary,"B109 Subhodaya Laurels,",15
4,3.554806e+12,gifting,B109 Subhodaya Laurels,13
...,...,...,...,...
49944,6.987795e+12,primary,"Room number 163, Opal Hostel, IITISM Dhanbad",1
49945,6.988563e+12,primary,"Flat no 282, Tower 2, Kalpataru Paramount, Balkum",1
49946,6.988957e+12,primary,KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ROAD,1
49947,6.989049e+12,primary,Zenith Youth Boys Hostel,1


In [92]:
result_df_tag = result_df[['user_id', 'address_type', 'delivery_address']]
result_df_tag

,user_id,address_type,delivery_address
0,2.912877e+12,primary,LODHA Aurum Grande. Kanjurmarg East
1,2.912877e+12,gifting,Electronics City Phase 1 Electronic City
2,2.912877e+12,gifting,"269/127 Birhana, Devi dayal Marg , Lucknow"
3,3.554806e+12,primary,"B109 Subhodaya Laurels,"
4,3.554806e+12,gifting,B109 Subhodaya Laurels
...,...,...,...
49944,6.987795e+12,primary,"Room number 163, Opal Hostel, IITISM Dhanbad"
49945,6.988563e+12,primary,"Flat no 282, Tower 2, Kalpataru Paramount, Balkum"
49946,6.988957e+12,primary,KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ROAD
49947,6.989049e+12,primary,Zenith Youth Boys Hostel


In [93]:
final_df_with_tagging = pd.merge(filtered_df, result_df_tag, on=["user_id", "delivery_address"], how="left")

In [94]:
final_df_with_tagging

,order_id,order_date,user_id,mrp,discount_percent,discount_amount,total_price,product_id,product_name,quantity,sp,product_category,delivery_address,address_type
0,5341764845706,2023-12-08,6.377996e+12,299.0,0.0,0.0,299.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,299.0,South Indian Filter Coffee,"Bungalow no 71, NEW MOTI BAGH, OPP \nHOTEL LEE...",gifting
1,5341763338378,2023-12-08,6.935222e+12,199.0,0.0,0.0,224.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,199.0,South Indian Filter Coffee,"34 BB street, bhadrakali, hooghly",primary
2,5341750689930,2023-12-08,6.704616e+12,598.0,0.0,0.0,598.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,2,299.0,South Indian Filter Coffee,"3/4 - 80 - EWS,",primary
3,5341738500234,2023-12-08,6.935186e+12,299.0,0.0,0.0,324.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,299.0,South Indian Filter Coffee,"21 Sneha flats, 4th Main Road, Kasturbanagar, ...",primary
4,5341737910410,2023-12-08,6.801344e+12,299.0,0.0,0.0,324.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,299.0,South Indian Filter Coffee,Address:3/336-1 404-1 venugopala Swamy Street ...,primary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83643,5382883868810,2024-01-07,6.989049e+12,1200.0,0.4,480.0,720.0,5.321648e+12,Single Malt Whiskey Barrel Aged Coffee - Frenc...,1,1200.0,Small Batch Series,Zenith Youth Boys Hostel,primary
83644,5382880723082,2024-01-07,6.692528e+12,1260.0,0.5,630.0,655.0,5.321622e+12,Peaberry Coffee - Medium Roast - Moka Pot / 25...,1,630.0,Single Origin Coffee,66A/2 GAUTAMNAGAR,primary
83645,5382880723082,2024-01-07,6.692528e+12,1260.0,0.5,630.0,655.0,6.624082e+12,Blueberry Mocha - Speciality Blend - 250 gm / ...,1,630.0,,66A/2 GAUTAMNAGAR,primary
83646,5382825181322,2024-01-07,6.988957e+12,199.0,0.0,0.0,199.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,199.0,South Indian Filter Coffee,KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ROAD,primary


In [95]:
final_df_with_tagging.to_csv("final_df_with_tagging.csv", index=False)

In [67]:
# lst = customer_address["Delivery_Address_Counts"].iloc[0][0]
# address = lst[0]
# count = lst[1]

In [68]:
# # Create a new DataFrame to store the extracted information
# new_dataframe = pd.DataFrame(columns=['user_id', 'Address', 'Count'])

# # Iterate through each row of the original DataFrame
# for index, row in customer_address.iterrows():
#     # Extract user_id from the row
#     user_id = row['user_id']

#     # Assuming each row contains a list of lists
#     for lst in row['Delivery_Address_Counts']:
#         # Extract address and count from the nested list
#         address = lst[0]
#         count = lst[1]

#         # Add a new row to the new DataFrame
#         new_dataframe = new_dataframe.append({'user_id': user_id, 'Address': address, 'Count': count}, ignore_index=True)

# # Display the new DataFrame
# print(new_dataframe)

In [69]:
# new_dataframe["user_id"].iloc[0]

In [70]:
# new_dataframe[new_dataframe["user_id"] == 2912876953738.0]

In [71]:
# tag_df = new_dataframe.groupby(["user_id", "customer_address"]).agg(
#     Tag =
# ).reset_index()

In [72]:
# customer_address["customer_address"].iloc[0]

In [73]:
# customer_address["Delivery_Address"].iloc[0]

In [98]:
import pandas as pd

def count_and_quote_unique(x):
    unique_values = pd.unique(x)
    quoted_values = list("'" + unique_values + "'")
    count = len(unique_values)
    return quoted_values, count

customer_address = data1.groupby("user_id").agg(
    # Customer_Address=pd.NamedAgg(column='customer_address', aggfunc=lambda x: count_and_quote_unique(x)),
    Delivery_Address=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: count_and_quote_unique(x)),
    Delivery_Address_Counts=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: x.value_counts().reset_index()),
).reset_index()

# Extract the counts and quoted values
# customer_address['Customer_Address'] = customer_address['Customer_Address'].apply(lambda x: x[0])
customer_address['Delivery_Address'] = customer_address['Delivery_Address'].apply(lambda x: x[0])


In [99]:
customer_address.to_csv("customer_address.csv", index=False)

In [100]:
customer_address["Delivery_Address_Counts"].iloc[0]

array([['LODHA Aurum Grande. Kanjurmarg East', 2],
       ['Electronics City Phase 1 Electronic City', 1],
       ['269/127 Birhana, Devi dayal Marg , Lucknow', 1]], dtype=object)

In [101]:
customer_address["Delivery_Address"].iloc[0]

["'Electronics City Phase 1 Electronic City'",
 "'LODHA Aurum Grande. Kanjurmarg East'",
 "'269/127 Birhana, Devi dayal Marg , Lucknow'"]

In [102]:
final_df_new_uplaod

,user_id,Order_First_Date,Order_Last_Date,Total_Order_Value,Total_Order_Counts,User_Journey,User_First_Tag,User_Final_Tag,Total_Quantity,Mean_Delta_Days_in_Orders,...,Average_Discount_Percentage,Average_Discount_List,AOV_List,Discount_Category,Suggested_Assortment,Order_Count_Without_Discount,Order_Count_With_Discount,No_of_SKUs,AOV_Category,Order_Count_Category
0,2.912877e+12,2021-07-09,2023-12-01,2304.0,4,"[New Users, Repeat Churned Users]",New Users,Repeat Churned Users,5,291.666667,...,0.285000,"[0.24, 0.5, 0.4, 0.0]","[444.6, 490.0, 294.0, 274.0]",Higher Discount,[[Attikan Estate Coffee - 250 gm / Whole Beans...,1,3,3,Higher AOV,Higher Orders
1,3.554806e+12,2020-10-15,2020-12-02,25880.0,35,"[Recent & Loyal Users, Champion Users]",Recent & Loyal Users,Repeat Churned Users,35,2.257143,...,0.996478,"[0.9981818181818182, 0.9993746091307066, 0.980...","[1.0, 12.0, 11.0]",Higher Discount,[Baba Budangiri Estate Coffee - Whole Beans / ...,0,8,33,Higher AOV,Higher Orders
2,3.555623e+12,2023-01-11,2023-01-11,2100.0,1,[New Users],New Users,Churned Users,1,NaN,...,0.600000,[0.6],[840.0],Higher Discount,[[Blueberry Mocha - Speciality Blend - 250 gm ...,0,1,1,Higher AOV,Lower Orders
3,3.555652e+12,2022-10-23,2022-10-23,1000.0,1,[New Users],New Users,Churned Users,2,NaN,...,0.275000,[0.275],[725.0],Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,2,Higher AOV,Lower Orders
4,3.555746e+12,2020-10-10,2022-10-28,31858.0,15,"[New Users, Champion Users, Repeat Churned Use...",New Users,Repeat Churned Users,20,53.428571,...,0.613038,"[0.0, 0.9981818181818182, 1.0, 0.2101105845181...","[585.0, 1.0, 440.0, 0.0, 9000.0, 1050.0, 12999.0]",Higher Discount,[[Thippanahalli Estate Coffee - Whole Beans / ...,1,3,7,Higher AOV,Higher Orders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44774,6.987795e+12,2024-01-06,2024-01-06,950.0,1,[New Users],New Users,New Users,1,NaN,...,0.400000,[0.4],[570.0],Higher Discount,[[Easy Brew (Hot Brew) - Mysore Nuggets - Pack...,0,1,1,Higher AOV,Lower Orders
44775,6.988563e+12,2024-01-07,2024-01-07,585.0,1,[New Users],New Users,New Users,1,NaN,...,0.400000,[0.4],[351.0],Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,1,Higher AOV,Lower Orders
44776,6.988957e+12,2024-01-07,2024-01-07,199.0,1,[New Users],New Users,New Users,1,NaN,...,0.000000,[0.0],[199.0],No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders
44777,6.989049e+12,2024-01-07,2024-01-07,1200.0,1,[New Users],New Users,New Users,1,NaN,...,0.400000,[0.4],[720.0],Higher Discount,[[Single Malt Whiskey Barrel Aged Coffee - Hom...,0,1,1,Higher AOV,Lower Orders


In [103]:
final_df_new_uplaod_with_address = pd.merge(final_df_new_uplaod, customer_address, on="user_id", how="left")

In [104]:
customer_address

,user_id,Delivery_Address,Delivery_Address_Counts
0,2.912877e+12,"['Electronics City Phase 1 Electronic City', '...","[[LODHA Aurum Grande. Kanjurmarg East, 2], [El..."
1,3.554806e+12,"['B109 Subhodaya Laurels,', '#14 Kalpavriksha ...","[[B109 Subhodaya Laurels,, 15], [B109 Subhoday..."
2,3.555623e+12,"['16B-305, Bollineni Hillside']","[[16B-305, Bollineni Hillside, 1]]"
3,3.555652e+12,"['3C, Lexus Apartments']","[[3C, Lexus Apartments, 2]]"
4,3.555746e+12,"['3G2, Maran Enclave, 23 West Vanniar Street,'...",[[g wing arkade earth kanjurmarg east nehru na...
...,...,...,...
44786,6.987795e+12,"['Room number 163, Opal Hostel, IITISM Dhanbad']","[[Room number 163, Opal Hostel, IITISM Dhanbad..."
44787,6.988563e+12,"['Flat no 282, Tower 2, Kalpataru Paramount, B...","[[Flat no 282, Tower 2, Kalpataru Paramount, B..."
44788,6.988957e+12,['KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ...,[[KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ...
44789,6.989049e+12,['Zenith Youth Boys Hostel'],"[[Zenith Youth Boys Hostel, 1]]"


In [105]:
final_df_new_uplaod_with_address

,user_id,Order_First_Date,Order_Last_Date,Total_Order_Value,Total_Order_Counts,User_Journey,User_First_Tag,User_Final_Tag,Total_Quantity,Mean_Delta_Days_in_Orders,...,AOV_List,Discount_Category,Suggested_Assortment,Order_Count_Without_Discount,Order_Count_With_Discount,No_of_SKUs,AOV_Category,Order_Count_Category,Delivery_Address,Delivery_Address_Counts
0,2.912877e+12,2021-07-09,2023-12-01,2304.0,4,"[New Users, Repeat Churned Users]",New Users,Repeat Churned Users,5,291.666667,...,"[444.6, 490.0, 294.0, 274.0]",Higher Discount,[[Attikan Estate Coffee - 250 gm / Whole Beans...,1,3,3,Higher AOV,Higher Orders,"['Electronics City Phase 1 Electronic City', '...","[[LODHA Aurum Grande. Kanjurmarg East, 2], [El..."
1,3.554806e+12,2020-10-15,2020-12-02,25880.0,35,"[Recent & Loyal Users, Champion Users]",Recent & Loyal Users,Repeat Churned Users,35,2.257143,...,"[1.0, 12.0, 11.0]",Higher Discount,[Baba Budangiri Estate Coffee - Whole Beans / ...,0,8,33,Higher AOV,Higher Orders,"['B109 Subhodaya Laurels,', '#14 Kalpavriksha ...","[[B109 Subhodaya Laurels,, 15], [B109 Subhoday..."
2,3.555623e+12,2023-01-11,2023-01-11,2100.0,1,[New Users],New Users,Churned Users,1,NaN,...,[840.0],Higher Discount,[[Blueberry Mocha - Speciality Blend - 250 gm ...,0,1,1,Higher AOV,Lower Orders,"['16B-305, Bollineni Hillside']","[[16B-305, Bollineni Hillside, 1]]"
3,3.555652e+12,2022-10-23,2022-10-23,1000.0,1,[New Users],New Users,Churned Users,2,NaN,...,[725.0],Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,2,Higher AOV,Lower Orders,"['3C, Lexus Apartments']","[[3C, Lexus Apartments, 2]]"
4,3.555746e+12,2020-10-10,2022-10-28,31858.0,15,"[New Users, Champion Users, Repeat Churned Use...",New Users,Repeat Churned Users,20,53.428571,...,"[585.0, 1.0, 440.0, 0.0, 9000.0, 1050.0, 12999.0]",Higher Discount,[[Thippanahalli Estate Coffee - Whole Beans / ...,1,3,7,Higher AOV,Higher Orders,"['3G2, Maran Enclave, 23 West Vanniar Street,'...",[[g wing arkade earth kanjurmarg east nehru na...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44774,6.987795e+12,2024-01-06,2024-01-06,950.0,1,[New Users],New Users,New Users,1,NaN,...,[570.0],Higher Discount,[[Easy Brew (Hot Brew) - Mysore Nuggets - Pack...,0,1,1,Higher AOV,Lower Orders,"['Room number 163, Opal Hostel, IITISM Dhanbad']","[[Room number 163, Opal Hostel, IITISM Dhanbad..."
44775,6.988563e+12,2024-01-07,2024-01-07,585.0,1,[New Users],New Users,New Users,1,NaN,...,[351.0],Higher Discount,[[Thippanahalli Estate Coffee - French Press /...,0,1,1,Higher AOV,Lower Orders,"['Flat no 282, Tower 2, Kalpataru Paramount, B...","[[Flat no 282, Tower 2, Kalpataru Paramount, B..."
44776,6.988957e+12,2024-01-07,2024-01-07,199.0,1,[New Users],New Users,New Users,1,NaN,...,[199.0],No Discount,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders,['KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ...,[[KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ...
44777,6.989049e+12,2024-01-07,2024-01-07,1200.0,1,[New Users],New Users,New Users,1,NaN,...,[720.0],Higher Discount,[[Single Malt Whiskey Barrel Aged Coffee - Hom...,0,1,1,Higher AOV,Lower Orders,['Zenith Youth Boys Hostel'],"[[Zenith Youth Boys Hostel, 1]]"


In [106]:
final_df_new_uplaod_with_address.to_csv("fdfnuwa_new.csv", index=False)

In [107]:
final_df_with_tagging

,order_id,order_date,user_id,mrp,discount_percent,discount_amount,total_price,product_id,product_name,quantity,sp,product_category,delivery_address,address_type
0,5341764845706,2023-12-08,6.377996e+12,299.0,0.0,0.0,299.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,299.0,South Indian Filter Coffee,"Bungalow no 71, NEW MOTI BAGH, OPP \nHOTEL LEE...",gifting
1,5341763338378,2023-12-08,6.935222e+12,199.0,0.0,0.0,224.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,199.0,South Indian Filter Coffee,"34 BB street, bhadrakali, hooghly",primary
2,5341750689930,2023-12-08,6.704616e+12,598.0,0.0,0.0,598.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,2,299.0,South Indian Filter Coffee,"3/4 - 80 - EWS,",primary
3,5341738500234,2023-12-08,6.935186e+12,299.0,0.0,0.0,324.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,299.0,South Indian Filter Coffee,"21 Sneha flats, 4th Main Road, Kasturbanagar, ...",primary
4,5341737910410,2023-12-08,6.801344e+12,299.0,0.0,0.0,324.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,299.0,South Indian Filter Coffee,Address:3/336-1 404-1 venugopala Swamy Street ...,primary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83643,5382883868810,2024-01-07,6.989049e+12,1200.0,0.4,480.0,720.0,5.321648e+12,Single Malt Whiskey Barrel Aged Coffee - Frenc...,1,1200.0,Small Batch Series,Zenith Youth Boys Hostel,primary
83644,5382880723082,2024-01-07,6.692528e+12,1260.0,0.5,630.0,655.0,5.321622e+12,Peaberry Coffee - Medium Roast - Moka Pot / 25...,1,630.0,Single Origin Coffee,66A/2 GAUTAMNAGAR,primary
83645,5382880723082,2024-01-07,6.692528e+12,1260.0,0.5,630.0,655.0,6.624082e+12,Blueberry Mocha - Speciality Blend - 250 gm / ...,1,630.0,,66A/2 GAUTAMNAGAR,primary
83646,5382825181322,2024-01-07,6.988957e+12,199.0,0.0,0.0,199.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,199.0,South Indian Filter Coffee,KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ROAD,primary


In [108]:

# Count the number of tags with 'primary' for each user_id
primary_count = final_df_with_tagging[final_df_with_tagging['address_type'] == 'primary'].groupby('user_id').size().reset_index(name='primary_count')

# Count the number of tags with 'gifting' for each user_id
gifting_count = final_df_with_tagging[final_df_with_tagging['address_type'] == 'gifting'].groupby('user_id').size().reset_index(name='gifting_count')

# Merge the two DataFrames on user_id
result = pd.merge(primary_count, gifting_count, on='user_id', how='outer').fillna(0)



In [109]:
result

,user_id,primary_count,gifting_count
0,2.912877e+12,2,2.0
1,3.554806e+12,15,28.0
2,3.555623e+12,1,0.0
3,3.555652e+12,2,0.0
4,3.555746e+12,9,6.0
...,...,...,...
44786,6.987795e+12,1,0.0
44787,6.988563e+12,1,0.0
44788,6.988957e+12,1,0.0
44789,6.989049e+12,1,0.0


In [121]:
final_df_final_new = pd.merge(final_df_new_uplaod_with_address, result, on="user_id", how="left")

In [111]:
final_df_final_new.to_csv("fdfn.csv", index=False)

In [124]:
final_df_final_new

,user_id,Order_First_Date,Order_Last_Date,Total_Order_Value,Total_Order_Counts,User_Journey,User_First_Tag,User_Final_Tag,Total_Quantity,Mean_Delta_Days_in_Orders,...,Suggested_Assortment,Order_Count_Without_Discount,Order_Count_With_Discount,No_of_SKUs,AOV_Category,Order_Count_Category,Delivery_Address,Delivery_Address_Counts,primary_count,gifting_count
0,2.912877e+12,2021-07-09,2023-12-01,2304.0,4,"[New Users, Repeat Churned Users]",New Users,Repeat Churned Users,5,291.666667,...,[[Attikan Estate Coffee - 250 gm / Whole Beans...,1,3,3,Higher AOV,Higher Orders,"['Electronics City Phase 1 Electronic City', '...","[[LODHA Aurum Grande. Kanjurmarg East, 2], [El...",2,2.0
1,3.554806e+12,2020-10-15,2020-12-02,25880.0,35,"[Recent & Loyal Users, Champion Users]",Recent & Loyal Users,Repeat Churned Users,35,2.257143,...,[Baba Budangiri Estate Coffee - Whole Beans / ...,0,8,33,Higher AOV,Higher Orders,"['B109 Subhodaya Laurels,', '#14 Kalpavriksha ...","[[B109 Subhodaya Laurels,, 15], [B109 Subhoday...",15,28.0
2,3.555623e+12,2023-01-11,2023-01-11,2100.0,1,[New Users],New Users,Churned Users,1,NaN,...,[[Blueberry Mocha - Speciality Blend - 250 gm ...,0,1,1,Higher AOV,Lower Orders,"['16B-305, Bollineni Hillside']","[[16B-305, Bollineni Hillside, 1]]",1,0.0
3,3.555652e+12,2022-10-23,2022-10-23,1000.0,1,[New Users],New Users,Churned Users,2,NaN,...,[[Thippanahalli Estate Coffee - French Press /...,0,1,2,Higher AOV,Lower Orders,"['3C, Lexus Apartments']","[[3C, Lexus Apartments, 2]]",1,0.0
4,3.555746e+12,2020-10-10,2022-10-28,31858.0,15,"[New Users, Champion Users, Repeat Churned Use...",New Users,Repeat Churned Users,20,53.428571,...,[[Thippanahalli Estate Coffee - Whole Beans / ...,1,3,7,Higher AOV,Higher Orders,"['3G2, Maran Enclave, 23 West Vanniar Street,'...",[[g wing arkade earth kanjurmarg east nehru na...,9,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44774,6.987795e+12,2024-01-06,2024-01-06,950.0,1,[New Users],New Users,New Users,1,NaN,...,[[Easy Brew (Hot Brew) - Mysore Nuggets - Pack...,0,1,1,Higher AOV,Lower Orders,"['Room number 163, Opal Hostel, IITISM Dhanbad']","[[Room number 163, Opal Hostel, IITISM Dhanbad...",1,0.0
44775,6.988563e+12,2024-01-07,2024-01-07,585.0,1,[New Users],New Users,New Users,1,NaN,...,[[Thippanahalli Estate Coffee - French Press /...,0,1,1,Higher AOV,Lower Orders,"['Flat no 282, Tower 2, Kalpataru Paramount, B...","[[Flat no 282, Tower 2, Kalpataru Paramount, B...",1,0.0
44776,6.988957e+12,2024-01-07,2024-01-07,199.0,1,[New Users],New Users,New Users,1,NaN,...,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders,['KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ...,[[KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ...,1,0.0
44777,6.989049e+12,2024-01-07,2024-01-07,1200.0,1,[New Users],New Users,New Users,1,NaN,...,[[Single Malt Whiskey Barrel Aged Coffee - Hom...,0,1,1,Higher AOV,Lower Orders,['Zenith Youth Boys Hostel'],"[[Zenith Youth Boys Hostel, 1]]",1,0.0


In [125]:
final_df_final_new[final_df_final_new["Total_Order_Counts"] == 1]

,user_id,Order_First_Date,Order_Last_Date,Total_Order_Value,Total_Order_Counts,User_Journey,User_First_Tag,User_Final_Tag,Total_Quantity,Mean_Delta_Days_in_Orders,...,Suggested_Assortment,Order_Count_Without_Discount,Order_Count_With_Discount,No_of_SKUs,AOV_Category,Order_Count_Category,Delivery_Address,Delivery_Address_Counts,primary_count,gifting_count
2,3.555623e+12,2023-01-11,2023-01-11,2100.0,1,[New Users],New Users,Churned Users,1,NaN,...,[[Blueberry Mocha - Speciality Blend - 250 gm ...,0,1,1,Higher AOV,Lower Orders,"['16B-305, Bollineni Hillside']","[[16B-305, Bollineni Hillside, 1]]",1,0.0
3,3.555652e+12,2022-10-23,2022-10-23,1000.0,1,[New Users],New Users,Churned Users,2,NaN,...,[[Thippanahalli Estate Coffee - French Press /...,0,1,2,Higher AOV,Lower Orders,"['3C, Lexus Apartments']","[[3C, Lexus Apartments, 2]]",1,0.0
5,3.555913e+12,2020-10-09,2020-10-09,575.0,1,[New Users],New Users,Churned Users,1,NaN,...,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Higher AOV,Lower Orders,"['504, Nilgiri, Lok Everest Complex, JSD Road,...","[[504, Nilgiri, Lok Everest Complex, JSD Road,...",1,0.0
8,3.559544e+12,2020-10-08,2020-10-08,630.0,1,[New Users],New Users,Churned Users,1,NaN,...,[[Thippanahalli Estate Coffee - French Press /...,1,0,1,Higher AOV,Lower Orders,['c 604 new friends colony '],"[[c 604 new friends colony , 1]]",1,0.0
9,3.562059e+12,2022-03-02,2022-03-02,2600.0,1,[New Users],New Users,Churned Users,4,NaN,...,[[Attikan Estate Coffee - 250 gm / Whole Beans...,0,1,2,Higher AOV,Lower Orders,"['117,Azad Nagar, Ayodhya Road']","[[117,Azad Nagar, Ayodhya Road, 3]]",1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44774,6.987795e+12,2024-01-06,2024-01-06,950.0,1,[New Users],New Users,New Users,1,NaN,...,[[Easy Brew (Hot Brew) - Mysore Nuggets - Pack...,0,1,1,Higher AOV,Lower Orders,"['Room number 163, Opal Hostel, IITISM Dhanbad']","[[Room number 163, Opal Hostel, IITISM Dhanbad...",1,0.0
44775,6.988563e+12,2024-01-07,2024-01-07,585.0,1,[New Users],New Users,New Users,1,NaN,...,[[Thippanahalli Estate Coffee - French Press /...,0,1,1,Higher AOV,Lower Orders,"['Flat no 282, Tower 2, Kalpataru Paramount, B...","[[Flat no 282, Tower 2, Kalpataru Paramount, B...",1,0.0
44776,6.988957e+12,2024-01-07,2024-01-07,199.0,1,[New Users],New Users,New Users,1,NaN,...,[[South Indian Traditional Filter Coffee - Dar...,1,0,1,Lower AOV,Lower Orders,['KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ...,[[KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ...,1,0.0
44777,6.989049e+12,2024-01-07,2024-01-07,1200.0,1,[New Users],New Users,New Users,1,NaN,...,[[Single Malt Whiskey Barrel Aged Coffee - Hom...,0,1,1,Higher AOV,Lower Orders,['Zenith Youth Boys Hostel'],"[[Zenith Youth Boys Hostel, 1]]",1,0.0


In [123]:
final_df_final_new["primary_count"] = final_df_final_new.apply(lambda row: 1 if row["Total_Order_Counts"] == 1 else row["primary_count"], axis=1)